In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql.functions import rand, when
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

StatementMeta(, bf21161b-2c40-41dd-8627-8452b81ac659, 3, Finished, Available, Finished)

In [2]:
df = spark.read.format("delta").load("Tables/gold_Customer360_sentiment")


StatementMeta(, bf21161b-2c40-41dd-8627-8452b81ac659, 4, Finished, Available, Finished)

In [5]:
df = df.withColumn("is_churned", when(rand() > 0.7, 1).otherwise(0))

StatementMeta(, bf21161b-2c40-41dd-8627-8452b81ac659, 7, Finished, Available, Finished)

In [6]:
indexer = StringIndexer(inputCol = "Sentiment", outputCol = "SentimentIndex")
df_indexed = indexer.fit(df).transform(df)

StatementMeta(, bf21161b-2c40-41dd-8627-8452b81ac659, 8, Finished, Available, Finished)

In [7]:
from pyspark.sql.functions import col
df_indexed = df_indexed.withColumn("PurchaseAmount", col("PurchaseAmount").cast("double"))

StatementMeta(, bf21161b-2c40-41dd-8627-8452b81ac659, 9, Finished, Available, Finished)

In [8]:
features = ["PurchaseAmount", "SentimentIndex"]
assembler = VectorAssembler(inputCols=features, outputCol="features")
df_final = assembler.transform(df_indexed)

StatementMeta(, bf21161b-2c40-41dd-8627-8452b81ac659, 10, Finished, Available, Finished)

In [9]:
train_data, test_data = df_final.randomSplit([0.7, 0.3], seed=42)


StatementMeta(, bf21161b-2c40-41dd-8627-8452b81ac659, 11, Finished, Available, Finished)

In [10]:
lr = LogisticRegression(featuresCol = "features", labelCol = "is_churned")
model = lr.fit(train_data)

StatementMeta(, bf21161b-2c40-41dd-8627-8452b81ac659, 12, Finished, Available, Finished)

In [11]:
predictions = model.transform(test_data)


StatementMeta(, bf21161b-2c40-41dd-8627-8452b81ac659, 13, Finished, Available, Finished)

In [13]:
evaluator = BinaryClassificationEvaluator(labelCol="is_churned")
auc = evaluator.evaluate(predictions)

StatementMeta(, bf21161b-2c40-41dd-8627-8452b81ac659, 15, Finished, Available, Finished)

IllegalArgumentException: requirement failed: rawPredictionCol vectors must have length=2, but got 1

In [18]:
print("Model AUC: ", auc)

StatementMeta(, ab20ebd8-a4b0-4936-9a1e-eef0193f5b21, 20, Finished, Available, Finished)

NameError: name 'auc' is not defined

In [14]:
predictions.write.format("delta").mode("overwrite").save("Tables/gold_customer360_churn_predictions")

StatementMeta(, bf21161b-2c40-41dd-8627-8452b81ac659, 16, Finished, Available, Finished)

AnalysisException: [_LEGACY_ERROR_TEMP_DELTA_0007] A schema mismatch detected when writing to the Delta table (Table ID: 5b1d7cb8-0b86-4daa-b137-6812c1d3c19b).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- CustomerID: string (nullable = true)
-- PurchaseAmount: double (nullable = true)
-- Sentiment: string (nullable = true)
-- prediction: double (nullable = true)
-- is_churned: integer (nullable = true)


Data schema:
root
-- CustomerID: string (nullable = true)
-- Name: string (nullable = true)
-- Email: string (nullable = true)
-- Location: string (nullable = true)
-- Gender: string (nullable = true)
-- Product: string (nullable = true)
-- PurchaseAmount: double (nullable = true)
-- TransactionDate: string (nullable = true)

In [15]:
display(predictions.select("CustomerID", "features", "is_churned", "prediction", "probability"))

StatementMeta(, bf21161b-2c40-41dd-8627-8452b81ac659, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 996db913-71ed-4146-90e9-2b0ace804c8f)

In [16]:
df_export = predictions.select("CustomerID", "PurchaseAmount", "Sentiment","prediction","is_churned")
df_export.write.format("delta").mode("overwrite").save("Tables/gold_customer360_churn_predictions")

StatementMeta(, bf21161b-2c40-41dd-8627-8452b81ac659, 18, Finished, Available, Finished)

In [17]:
df_final = df_final.select("CustomerID", "ChurnPrediction","is_churned")  
df_final.write.format("delta").mode("overwrite").save("Tables/gold_customer360_churn_predictions_clean")

StatementMeta(, bf21161b-2c40-41dd-8627-8452b81ac659, 19, Finished, Available, Finished)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `ChurnPrediction` cannot be resolved. Did you mean one of the following? [`Product`, `Location`, `CustomerID`, `Gender`, `PurchaseAmount`].;
'Project [CustomerID#756, 'ChurnPrediction, is_churned#1258]
+- Project [CustomerID#756, Name#757, Email#758, Location#759, Gender#760, Product#761, PurchaseAmount#1310, TransactionDate#763, ReviewText#764, ReviewDate#765, Sentiment#766, is_churned#1258, SentimentIndex#1293, UDF(struct(PurchaseAmount, PurchaseAmount#1310, SentimentIndex, SentimentIndex#1293)) AS features#1325]
   +- Project [CustomerID#756, Name#757, Email#758, Location#759, Gender#760, Product#761, cast(PurchaseAmount#762 as double) AS PurchaseAmount#1310, TransactionDate#763, ReviewText#764, ReviewDate#765, Sentiment#766, is_churned#1258, SentimentIndex#1293]
      +- Project [CustomerID#756, Name#757, Email#758, Location#759, Gender#760, Product#761, PurchaseAmount#762, TransactionDa

In [18]:
df_final.printSchema()

StatementMeta(, bf21161b-2c40-41dd-8627-8452b81ac659, 20, Finished, Available, Finished)

root
 |-- CustomerID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- PurchaseAmount: double (nullable = true)
 |-- TransactionDate: string (nullable = true)
 |-- ReviewText: string (nullable = true)
 |-- ReviewDate: string (nullable = true)
 |-- Sentiment: string (nullable = true)
 |-- is_churned: integer (nullable = false)
 |-- SentimentIndex: double (nullable = false)
 |-- features: vector (nullable = true)



In [19]:
df_final_churn = df_final.selectExpr("CustomerID", "is_churned as ChurnPrediction")

df_final_churn.write.format("delta").mode("overwrite").save("Tables/gold_customer360_churn_predictions_clean")


StatementMeta(, bf21161b-2c40-41dd-8627-8452b81ac659, 21, Finished, Available, Finished)

In [20]:
df_final_churn.write.mode("overwrite").option("header", "true").csv("Files/gold_churn_predictions_csv")


StatementMeta(, bf21161b-2c40-41dd-8627-8452b81ac659, 22, Finished, Available, Finished)

In [22]:
df_result = df.select(
    "CustomerID",
    "Gender",  
    "Location",
    "Product",
    "PurchaseAmount",
    "Sentiment",
    "is_churned"
)


StatementMeta(, bf21161b-2c40-41dd-8627-8452b81ac659, 24, Finished, Available, Finished)

In [24]:
df_result.write.format("delta").option("mergeSchema", "true").mode("overwrite").save("Tables/gold_customer360_churn_predictions_clean")


StatementMeta(, bf21161b-2c40-41dd-8627-8452b81ac659, 26, Finished, Available, Finished)

In [25]:
df_result.printSchema()
df_result.show(5)


StatementMeta(, bf21161b-2c40-41dd-8627-8452b81ac659, 27, Finished, Available, Finished)

root
 |-- CustomerID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- PurchaseAmount: string (nullable = true)
 |-- Sentiment: string (nullable = true)
 |-- is_churned: integer (nullable = false)

+----------+------+---------+-------+--------------+---------+----------+
|CustomerID|Gender| Location|Product|PurchaseAmount|Sentiment|is_churned|
+----------+------+---------+-------+--------------+---------+----------+
|      C001|     F|  Toronto|  Mouse|            40| Positive|         0|
|      C001|     F|  Toronto| Laptop|          1200| Positive|         0|
|      C002|     M|Vancouver| Tablet|           600| Negative|         0|
|      C002|     M|Vancouver|  Phone|           800| Negative|         0|
|      C003|     M|  Calgary|Monitor|           300|  Neutral|         0|
+----------+------+---------+-------+--------------+---------+----------+
only showing top 5 rows



In [26]:
df_result.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/gold_customer360_churn_predictions_clean")


StatementMeta(, bf21161b-2c40-41dd-8627-8452b81ac659, 28, Finished, Available, Finished)